In [1]:
!pip install -q peft==0.8.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 7.3 MB/s eta 0:00:00


In [2]:
import re

def clean_text(text):
    # replace all the special characters with space
    text=text.replace('+',' plus ')
    text=text.replace('-',' minus ')
    text=text.replace('*',' times ')
    text=text.replace('^',' power ')
    text=text.replace('/',' divide ')
    text=text.replace('<',' le ')
    text=text.replace('>',' ge ')
    text=text.replace('≤',' leq ')
    text=text.replace('<=',' leq ')
    text=text.replace('≥',' geq ')
    text=text.replace('>=',' geq ')
    text=text.replace('=',' equal ')
    text=text.replace('#',' numerical ')

    text = re.sub(r'[_\'\"]', '', text)

    text = re.sub(r'\W', ' ', text)
    return text

In [3]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer, TrainingArguments

model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
df = pd.read_json('dataset.json').replace(np.nan, 0)
texts=df["statement"].apply(str.lower).apply(clean_text)
texts

,statement
0,king kog got annoyed of the usual laxity of hi...
1,international coding procedures company icpc w...
2,consider a permutation p1 p2 dots pn of intege...
3,berland state university invites people from a...
4,you are given a positive integer n find a sequ...
...,...
9548,there are n houses along the road where anya l...
9549,misha didnt do his math homework for todays le...
9550,vasya had an array of n integers each element ...
9551,santa has prepared boxes with presents for n k...


In [5]:
# Get all columns that start with 'tags.'
tag_columns = [col for col in df.columns if col.startswith('tags.')]
print('tag_columns:', tag_columns)
print('tag_columns count:', len(tag_columns))
# Get the dataframe data in those columns

tag_columns: ['tags.data structures', 'tags.greedy', 'tags.implementation', 'tags.combinatorics', 'tags.math', 'tags.dp', 'tags.trees', 'tags.bitmasks', 'tags.constructive algorithms', 'tags.interactive', 'tags.sortings', 'tags.brute force', 'tags.strings', 'tags.geometry', 'tags.flows', 'tags.graphs', 'tags.binary search', 'tags.hashing', 'tags.probabilities', 'tags.number theory', 'tags.dsu', 'tags.shortest paths', 'tags.games', 'tags.string suffix structures', 'tags.dfs and similar', 'tags.two pointers', 'tags.divide and conquer', 'tags.graph matchings', 'tags.ternary search', 'tags.*special', 'tags.chinese remainder theorem', 'tags.fft', 'tags.meet-in-the-middle', 'tags.matrices', 'tags.2-sat', 'tags.expression parsing', 'tags.schedules']
tag_columns count: 37


In [6]:
super_categories = {
    'implementation': [
        'tags.implementation', 'tags.constructive algorithms'
    ],
    'maths': [
        'tags.math', 'tags.geometry', 'tags.matrices', 'tags.combinatorics', 'tags.probabilities',
        'tags.number theory', 'tags.chinese remainder theorem', 'tags.fft',
    ],
    'graphs': [
        'tags.trees', 'tags.dsu', 'tags.shortest paths', 'tags.data structures',
        'tags.dfs and similar', 'tags.flows', 'tags.graphs', 'tags.graph matchings',
    ],
    'optimization': [
        'tags.greedy', 'tags.dp', 'tags.two pointers', 'tags.sortings', 'tags.brute force',
        'tags.binary search', 'tags.divide and conquer', 'tags.meet-in-the-middle',
    ],
    'special-strings': [
        'tags.*special', 'tags.2-sat', 'tags.ternary search', 'tags.schedules', 'tags.games',
        'tags.bitmasks', 'tags.interactive', 'tags.strings', 'tags.string suffix structures',
        'tags.hashing', 'tags.expression parsing',
    ]
}

supertag_columns = [f'supertag.{key}' for key in super_categories]
for key in supertag_columns:
    df[key] = df[super_categories[key.split('.')[1]]].any(axis=1)
df[supertag_columns] = df[supertag_columns].astype(float)
df[supertag_columns]

,supertag.implementation,supertag.maths,supertag.graphs,supertag.optimization,supertag.special-strings
0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...
9548,1.0,0.0,0.0,0.0,0.0
9549,0.0,0.0,0.0,0.0,1.0
9550,1.0,0.0,0.0,0.0,0.0
9551,1.0,0.0,1.0,0.0,0.0


In [7]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(df['statement'].values.tolist(), df[supertag_columns].values.tolist(), test_size=.2)

In [8]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=500)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [9]:
class CFDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CFDataset(train_encodings, train_labels)
val_dataset = CFDataset(val_encodings, val_labels)

In [10]:
len(train_dataset)

7642

In [11]:
# Cargar el modelo preentrenado y el tokenizer

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(supertag_columns),problem_type = "multi_label_classification"
)
model.to('cuda')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [16]:
import peft
from peft import LoraConfig, get_peft_model, PeftModel

lora_config = LoraConfig(
    r=2,
    lora_alpha=16,
    target_modules=["q_lin", "k_lin","v_lin"],
)

peft_model = get_peft_model(model, lora_config)
peft_model.to('cuda')

PeftModel(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=2, bias=False)
              

In [17]:
training_args = TrainingArguments(
    output_dir='./distilbert-finetunned',
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=1,
    save_steps=1000,
    save_total_limit=1,                    # Mantener solo 1 versión del modelo guardado
    logging_steps=50,                      # Intervalos de logging menos frecuentes
    logging_dir="./logs",                  # Carpeta para los logs
    fp16=False,                            # Puedes probar a activarlo si tu GPU soporta fp16
    evaluation_strategy="no",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Step,Training Loss
50,0.697800
100,0.689400
150,0.688100
200,0.677800
250,0.649800
300,0.650000
350,0.630500
400,0.653200
450,0.628600
500,0.608600


Step,Training Loss
50,0.697800
100,0.689400
150,0.688100
200,0.677800
250,0.649800
300,0.650000
350,0.630500
400,0.653200
450,0.628600
500,0.608600


TrainOutput(global_step=15284, training_loss=0.6188235714684541, metrics={'train_runtime': 681.1155, 'train_samples_per_second': 22.44, 'train_steps_per_second': 22.44, 'total_flos': 1979820627852000.0, 'train_loss': 0.6188235714684541, 'epoch': 2.0})

In [19]:
trainer.evaluate()

{'eval_runtime': 33.6006,
 'eval_samples_per_second': 56.874,
 'eval_steps_per_second': 7.113,
 'epoch': 2.0}

In [20]:
new_texts = ["Given a reference of a node in a connected undirected graph. Return a deep copy (clone) of the graph. Each node in the graph contains a value (int) and a list (List[Node]) of its neighbors. For simplicity, each node's value is the same as the node's index (1-indexed). For example, the first node with val == 1, the second node with val == 2, and so on. The graph is represented in the test case using an adjacency list.An adjacency list is a collection of unordered lists used to represent a finite graph. Each list describes the set of neighbors of a node in the graph. The given node will always be the first node with val = 1. You must return the copy of the given node as a reference to the cloned graph.",
             "You are given a n m field consisting only of periods and asterisks Your task is to count all right triangles with two sides parallel to the square sides whose vertices are in the centers of cells A right triangle is a triangle in which one angle is a right angle that is a 90 degree angle The first line contains two positive integer numbers n and m 1 n m 1000 The following n lines consist of m characters each describing the field Only and are allowed Output a single number total number of square triangles in the field Please do not use lld specificator to read or write 64bit integers in C It is preffered to use cout also you may use I64d"]
inputs = tokenizer(new_texts, return_tensors='pt', padding=True, truncation=True).to('cuda')

with torch.no_grad():
    outputs = model(**inputs)

    predictions = torch.sigmoid(outputs.logits)
    # predictions = torch.argmax(outputs.logits, dim=-1)
for prediction in predictions:
  my_preds = []
  for pred, tag in zip(prediction, supertag_columns):
      my_preds.append((pred, tag))
  for pred, tag in sorted(my_preds, key=lambda x: x[0], reverse=True)[:5]:
      print(f"{tag}: {pred}")
  print()


supertag.optimization: 0.6676839590072632
supertag.implementation: 0.4131007194519043
supertag.maths: 0.40451475977897644
supertag.graphs: 0.25094106793403625
supertag.special-strings: 0.2168588489294052

supertag.optimization: 0.5601961016654968
supertag.implementation: 0.4610591530799866
supertag.maths: 0.39922407269477844
supertag.special-strings: 0.3043709993362427
supertag.graphs: 0.24439072608947754

